<a href="https://colab.research.google.com/github/ijusplab/criar-resumo/blob/main/Criar_Resumo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **10ª Turma Recursal**
## Criar Resumo de Pauta
(v. 1.0.0)

## **Instruções**

**INSTRUÇÕES**

1) Execute a célula `CARREGAR FERRAMENTA`. 

2) Execute a célula `CARREGAR DADOS DO SHAREPOINT`. 

3) Assim que o código for inicializado e aparecer o botão de `upload`, carregue o arquivo `csv` do sharepoint. Você verá uma prévia do conteúdo extraído logo abaixo da célula, em forma de tabela.

4) Execute a célula `GERAR ARQUIVO JSON`. O arquivo será baixado no seu browser em poucos segundos.


## **GERAR JSON**

In [ ]:
#@title **CARREGAR FERRAMENTA** { vertical-output: true }
MODO_DEPURACAO = False

import re, os, json, csv, pprint
from enum import Enum
from google.colab import files
import pandas as pd
import numpy as np
from IPython.display import display, HTML

##
## Obter o template html para o link
## Pela opção -O, o arquivo será sempre sobrescrito
##

!wget -O sharepoint.html https://raw.githubusercontent.com/ijusplab/criar-resumo/main/sharepoint.html
with open ('sharepoint.html') as file:
  link_template = file.read()

##
## Enums
##

class TIPO_VOTO(Enum):
  QUESTAO_DE_ORDEM = 1
  VOTO_VISTA = 2
  ED = 3
  MS = 4
  RECURSO_MC = 5
  RECURSO_EXECUCAO = 6
  RECURSO_SENTENCA = 7
  RETRATACAO = 8
  READEQUACAO = 9
  AGRAVO = 10
  VOTO_VENCEDOR = 11
  VOTO_VOGAL = 12

class STATUS_JULGAMENTO(Enum):
  EM_JULGAMENTO = 1
  RETIRADO = 2
  ADIADO = 3
  VISTA = 4
  SUSPENSO = 5

class SITUACAO_PAUTA(Enum):
  PAUTADO = 1
  NAO_PAUTADO = 2
  EM_MESA = 3
  PENDENTE = 4

##
## Mensagens
##

class mensageria:
  __END      = '\33[0m'
  __BOLD     = '\33[1m'
  __ITALIC   = '\33[3m'
  __URL      = '\33[4m'
  __BLINK    = '\33[5m'
  __BLINK2   = '\33[6m'
  __SELECTED = '\33[7m'

  __BLACK  = '\33[30m'
  __RED    = '\33[31m'
  __GREEN  = '\33[32m'
  __YELLOW = '\33[33m'
  __BLUE   = '\33[34m'
  __VIOLET = '\33[35m'
  __BEIGE  = '\33[36m'
  __WHITE  = '\33[37m'

  __BLACKBG  = '\33[40m'
  __REDBG    = '\33[41m'
  __GREENBG  = '\33[42m'
  __YELLOWBG = '\33[43m'
  __BLUEBG   = '\33[44m'
  __VIOLETBG = '\33[45m'
  __BEIGEBG  = '\33[46m'
  __WHITEBG  = '\33[47m'

  __GREY    = '\33[90m'
  __RED2    = '\33[91m'
  __GREEN2  = '\33[92m'
  __YELLOW2 = '\33[93m'
  __BLUE2   = '\33[94m'
  __VIOLET2 = '\33[95m'
  __BEIGE2  = '\33[96m'
  __WHITE2  = '\33[97m'

  __GREYBG    = '\33[100m'
  __REDBG2    = '\33[101m'
  __GREENBG2  = '\33[102m'
  __YELLOWBG2 = '\33[103m'
  __BLUEBG2   = '\33[104m'
  __VIOLETBG2 = '\33[105m'
  __BEIGEBG2  = '\33[106m'
  __WHITEBG2  = '\33[107m'

  @classmethod
  def sucesso(cls, msg):
    print(f'{cls.__BLUE}{cls.__BOLD}{msg}{cls.__END}')

  @classmethod
  def falha(cls, msg):
    print(f'{cls.__REDBG}{cls.__WHITE}{msg}{cls.__END}')

  @classmethod
  def alerta(cls, msg):
    print(f'{cls.__YELLOWBG}{msg}{cls.__END}')

  @classmethod
  def ok(cls, msg):
    print(f'{cls.__GREEN}{cls.__BOLD}{msg}{cls.__END}')

  @classmethod
  def normal(cls, msg):
    print(f'{cls.__BOLD}{msg}{cls.__END}')

  @classmethod
  def mostrar_objeto(cls, obj):
    printer = pprint.PrettyPrinter(indent=4)
    printer.pprint(obj)

##
## Para normalização dos nomes de colunas
##
class utils:
  __acentos = {
    'a': re.compile('[áàäãâ]'), 
    'e': re.compile('[éèëê]'), 
    'i': re.compile('[íìïî]'),
    'o': re.compile('[óòöõô]'),
    'u': re.compile('[úùüû]'),
    'c': re.compile('[ç]'),
    'A': re.compile('[ÁÀÄÃÂ]'), 
    'E': re.compile('[ÉÈËÊ]'), 
    'I': re.compile('[ÍÌÏÎ]'),
    'O': re.compile('[ÓÒÖÕÔ]'),
    'U': re.compile('[ÚÙÜÛ]'),
    'C': re.compile('[Ç]')
  }
  __caracteres_especiais = re.compile('[\(\)\[\]\{\}!@#$%¨&*+=ªº°§]')

  @classmethod
  def remover_acentuacao(cls, s):
    for letra, padrao in cls.__acentos.items():
      s = re.sub(padrao, letra, s)
    return s

  @classmethod
  def limpar(cls, s):
    s = cls.remover_acentuacao(s).upper().strip()
    s = re.sub(cls.__caracteres_especiais, '', s).strip()
    s = re.sub(r'\s{2,}', ' ', s).strip()
    return s

  @classmethod
  def normalizar(cls, s):
    s = cls.remover_acentuacao(s).upper().strip()
    s = re.sub(r'\d+_+', '', s)
    s = re.sub(r'\s+', '_', s)
    return s

##
## Extração dos dados do sharepoint
##
class Sharepoint:
  _instance = None

  def __init__(self):
    self.__resumo = None
    self.__resumo_antigo = None
    self.__processos = 0
    self.__grupos = []
    self.__votos = None
    self.__pastas = None
    self.__cadeira = ''
    self.__data = ''
    self.__complemento = ''

    self._cols = {
      "PROCESSO": "PROCESSO",
      "OBJETO": "OBJETO",
      "ALEGACOES": "ALEGACOES",
      "FUNDAMENTACAO": "FUNDAMENTACAO",
      "DISPOSITIVO": "DISPOSITIVO",
      "OBSERVACOES": "OBSERVACOES",
      "STATUS_JULGAMENTO": "STATUS_JULGAMENTO",
      "SITUACAO_PAUTA": "SITUACAO_PAUTA",
      "TIPO_VOTO": "TIPO_VOTO",
      "{IsFolder}": "IS_FOLDER",
      "{Link}": "LINK",
      "{Name}": "NAME",
      "{Path}": "PATH",
      "{FullPath}": "FULL_PATH",
      "OData__dlc_DocId": "ID",
      "OData__dlc_DocIdUrl": "PERMALINK"
    }

  @classmethod
  def instance(cls):
      if cls._instance is None:
          cls._instance = cls()
      return cls._instance

  def __extrair_cadeira(self, s):
    pattern = re.compile('Cadeira\s(\d+)')
    match = pattern.search(s)
    if not match:
      return ''
    return match.group(1)

  def __transformar_nome_coluna(self, s):
    if s in self._cols:
      return self._cols[s]
    return s

  def __extrair_valor(self, s):
    pattern = re.compile('\"Value\":\"(.+)\"')
    match = pattern.search(s)
    if not match:
      return s
    return match.group(1)

  def __extrair_numero_processo(self, s):
    pattern = re.compile('\d{7}-\d{2}\.\d{4}\.\d\.\d{2}\.\d{4}')
    match = pattern.search(s)
    if not match:
      return ''
    return match.group(0)

  # ajusta o link para abrir o documento no browser
  # no lugar do download
  def __criar_link_para_browser(self, link):
    if '?' in link:
      return f'{link}&web=1'
    return f'{link}?web=1'

  # obtem grupo do voto (a partir das duas pastas acima)
  # a partir de /pasta1/pasta2/arquivo, retorna pasta1_pasta2 
  def __obter_grupo(self, s):
    pattern = re.compile('.*\/([^\/]+\/[^\/]+)\/')
    match = pattern.search(s)
    if not match:
      return ''
    return match.group(1).replace('/', '_')

  def __obter_dic_processos_antigo(self, df):
    dic = {}
    for index, row in df.iterrows():
      key = re.sub('\D', '', row['PROCESSO'])
      dic[key] = {}
      dic[key]['so'] = False
      dic[key]['adv'] = ''
      dic[key]['concl'] = False
      dic[key]['jimp'] = ''
    return dic

  def __obter_usuarios_fake(self):
    usuarios = []
    nomes = ["Fulano", "Beltrano", "Sicrano", "Fulaninho"]
    siglas = ["F", "B", "S", "FN"]
    emails = ["x@gmail.com", "y@gmail.com", "w@gmail.com", "z@gmail.com"]
    for i in range(4):
      usuario = {}
      usuario["id"] = str(i + 1)
      usuario["nome"] = nomes[i]
      usuario["sigla"] = siglas[i]
      usuario["email"] = emails[i]
      usuario["sexo"] = "M"
      usuario["cadeira"] = "29"
      usuario["turma"] = "10"
      usuario["nivel"] = 1
      usuario["atribuicao"] = "Assessor"
      usuario["admin"] = True
      usuarios.append(usuario)
    return usuarios

  def __obter_sessao_fake(self, usuarios):
    sessao = {}
    chave = f'{self.__cadeira}{self.__data}_{self.__complemento}'
    sessao[chave] = {}
    sessao[chave]["r"] = usuarios[0]["email"]
    sessao[chave]["ri"] = usuarios[0]["id"]
    sessao[chave]["j1"] = usuarios[1]["email"]
    sessao[chave]["j1i"] = usuarios[1]["id"]
    sessao[chave]["j2"] = usuarios[2]["email"]
    sessao[chave]["j2i"] = usuarios[2]["id"]
    sessao[chave]["s"] = usuarios[3]["email"]
    sessao[chave]["si"] = usuarios[3]["id"]
    return sessao

  def __obter_titulos_fake(self):
    titulos = []
    titulo = {}
    titulo["titulo"] = self.__complemento
    titulo["item"] = "1"
    titulos.append(titulo)
    return titulos

  def __obter_dic_processos(self, df):
    dic = {}
    for index, row in df.iterrows():
      key = re.sub('\D', '', row['PROCESSO'])
      # a = advogado
      # t = tipo
      # so = so
      # pr = prioridade
      # ji = ji
      # s = situação
      # r = resultado
      # cr = complemento resultado
      # st = status
      # cst = complemento status
      # concl = concl
      # sa = sem acórdão
      dic[key] = {}
      dic[key]['a'] = ''
      dic[key]['t'] = re.sub('\D', '', row['TIPO_VOTO'])
      dic[key]['so'] = False
      dic[key]['pr'] = False
      dic[key]['ji'] = ''
      dic[key]['s'] = re.sub('\D', '', row['SITUACAO_PAUTA'])
      dic[key]['st'] = re.sub('\D', '', row['STATUS_JULGAMENTO'])
      dic[key]['cst'] = ''
      dic[key]['r'] = ''
      dic[key]['cr'] = ''
      dic[key]['concl'] = False
      dic[key]['sa'] = False
    return dic

  def __obter_grupos_ordenados(self, df_pastas):
    list = []
    for index, row in df_pastas.iterrows():
      dic = {}
      dic['titulo'] = row['GRUPO']
      dic['status'] = re.sub('\D', '', row['STATUS_JULGAMENTO'])
      dic['situacao'] = re.sub('\D', '', row['SITUACAO_PAUTA'])
      dic['tipoVoto'] = re.sub('\D', '', row['TIPO_VOTO'])
      dic['observacoes'] = row['OBSERVACOES']
      dic['dispositivo'] = row['DISPOSITIVO']
      link = self.__criar_link_para_browser(row['LINK'])
      dic['link'] = link
      dic['linkBtn'] = re.sub('\{\{link\}\}', link, link_template) 
      dic['downloadLink'] = row['LINK'] 
      lote = len(row['PROCESSO'].strip().split('\n'))
      dic['lote'] = [] if len(row['PROCESSO'].strip()) == 0 else lote
      list.append(dic)
    pattern = re.compile('^\d+$')
    list.sort(key=lambda item: 1000 if not pattern.search(item['tipoVoto']) else int(item['tipoVoto']))
    return list

  def __obter_list_resumo(self, df_arquivos, grupos_ordenados):
    agrupados = df_arquivos.groupby('GRUPO')
    resumo = []
    contador = 1

    for grupo in grupos_ordenados:
      try:
        items = agrupados.get_group(grupo['titulo'])
      except KeyError:
        continue
      if len(items) == 0:
        continue

      raiz = {}
      titulo = grupo['titulo']
      raiz['titulo'] = f'{contador}.{titulo}'
      raiz['observacoes'] = grupo['observacoes']
      raiz['link'] = grupo['link']
      raiz['linkBtn'] = grupo['linkBtn']
      raiz['downloadLink'] = grupo['downloadLink']
      raiz['tipo'] = 'lote' if len(grupo['lote']) > 0 else 'normal'
      raiz['status'] = grupo['status']
      raiz['situacao'] = grupo['situacao']
      raiz['tipoVoto'] = grupo['tipoVoto']

      col_link = {}
      col_link['numero'] = 1
      col_link['tamanho'] = 5
      col_link['nome'] = '🔗'
      col_alegacoes = {}
      col_alegacoes['numero'] = 2
      col_alegacoes['tamanho'] = 30
      col_alegacoes['nome'] = 'Alegações Recursais'
      col_fundamentacao = {}
      col_fundamentacao['numero'] = 3
      col_fundamentacao['tamanho'] = 45
      col_fundamentacao['nome'] = 'Fundamentação'
      col_resultado = {}
      col_resultado['numero'] = 2
      col_resultado['tamanho'] = 20
      col_resultado['nome'] = 'Resultado'
      raiz['colunas'] = [col_link, col_alegacoes, col_fundamentacao, col_resultado]

      processos = []
      subcontador = 1
      for index, row in items.iterrows():
        processo = {}
        processo['numero'] = re.sub('\D', '', row['PROCESSO'])
        processo['item'] = f'{contador}.{subcontador}'
        processo['ordem'] = f'{str(contador).zfill(3)}{str(subcontador).zfill(3)}'
        processo['status'] = re.sub('\D', '', row['STATUS_JULGAMENTO'])
        processo['situacao'] = re.sub('\D', '', row['SITUACAO_PAUTA'])
        processo['tipoVoto'] = re.sub('\D', '', row['TIPO_VOTO'])
        link = self.__criar_link_para_browser(row['LINK'])
        processo['link'] = link
        processo['linkBtn'] = re.sub('\{\{link\}\}', link, link_template) 
        processo['downloadLink'] = row['LINK'] 
        col_link = {}
        col_link['numero'] = 1
        col_link['tamanho'] = 5
        col_link['conteudo'] = processo['linkBtn']
        col_alegacoes = {}
        col_alegacoes['numero'] = 2
        col_alegacoes['tamanho'] = 30
        col_alegacoes['conteudo'] = row['ALEGACOES']
        col_fundamentacao = {}
        col_fundamentacao['numero'] = 3
        col_fundamentacao['tamanho'] = 45
        col_fundamentacao['conteudo'] = row['FUNDAMENTACAO']
        col_resultado = {}
        col_resultado['numero'] = 2
        col_resultado['tamanho'] = 20
        resultado = row['DISPOSITIVO'] or grupo['dispositivo']
        observacoes = row['OBSERVACOES']
        col_resultado['conteudo'] = f'{resultado}\n{observacoes}'
        processo['colunas'] = [col_link, col_alegacoes, col_fundamentacao, col_resultado]
        processos.append(processo)
        subcontador += 1

      raiz['processos'] = processos
      resumo.append(raiz)
      contador += 1
    
    return resumo

  def __ajustar_padroes(self):

    for item in self.__resumo['resumo']:
      status_default = item['status']
      situacao_default = item['situacao']
      tipo_default = item['tipoVoto']

      for processo in item['processos']:
        numero = processo['numero']
        status = self.__resumo['processos'][numero]['st'] 
        situacao = self.__resumo['processos'][numero]['s'] 
        tipo = self.__resumo['processos'][numero]['t'] 
        self.__resumo['processos'][numero]['st'] = status or status_default 
        self.__resumo['processos'][numero]['s'] = situacao or situacao_default
        self.__resumo['processos'][numero]['t'] = tipo or tipo_default

  def load(self):
    uploaded = files.upload()
    arquivos = list(uploaded.keys())
    print(arquivos)
    if len(arquivos) > 0:
      nome_arquivo = arquivos[0]
      conteudo = uploaded[nome_arquivo] 
    else:
      mensageria.falha('\n>>>> Nenhum arquivo selecionado')
      return

    if re.match('^.+\.(csv)$', nome_arquivo):

      # converte csv em dataframe, mantendo somente as
      # colunas desejadas
      df = pd.read_csv(nome_arquivo, sep=',', encoding='utf-8')
      df = df.fillna('')
      df.drop(df.columns.difference(list(self._cols.keys())), 1, inplace=True)
      df.columns = map(self.__transformar_nome_coluna, df.columns)

      # Extrai o valor das colunas customizadas do Sharepoint
      custom = ['PROCESSO', 'OBJETO', 'ALEGACOES', 'FUNDAMENTACAO', 'DISPOSITIVO', 'OBSERVACOES', 'STATUS_JULGAMENTO', 'SITUACAO_PAUTA', 'TIPO_VOTO']
      for col_name in custom:
        if col_name not in df.columns:
          df[col_name] = pd.Series()
          df[col_name] = df[col_name].fillna('')
        else:
          df[col_name] = df[col_name].apply(self.__extrair_valor)

      # Obtem dados da cadeira e do resumo
      self.__cadeira = self.__extrair_cadeira(df.at[0, 'PATH'])
      data = nome_arquivo[0:10].split('-')
      data.reverse()
      self.__data = ''.join(data)
      partes = nome_arquivo.replace('.csv', '').split('_')
      if len(partes) > 1:
        self.__complemento = partes[1]
      else:
        self.__complemento = 'Genérico'


      # Separa pastas e arquivos
      grouped = df.groupby(df['IS_FOLDER'])
      pastas = grouped.get_group(True).copy()
      arquivos = grouped.get_group(False).copy()

      # Identifica as divisões do resumo a partir das pastas
      pastas['GRUPO'] = pastas['FULL_PATH'].apply(lambda x: f'{x}/').apply(self.__obter_grupo)
      grupos = self.__obter_grupos_ordenados(pastas)

      # cria coluna com números de processos
      # e identifica se há nomes de arquivos inválidos
      arquivos['PROCESSO'] = arquivos['NAME'].apply(self.__extrair_numero_processo)
      try:
        errors = arquivos.groupby(arquivos['PROCESSO']).get_group('')['NAME'].tolist()
      except:
        errors = []

      if len(errors) > 0:
        errors = '\n'.join(errors)
        mensageria.falha(f'\n>>>> Os seguintes arquivos têm nome irregular:\n {errors}')
        return

      # cria coluna com a localização dos processos
      # e identifica se há localizações inválidas
      arquivos['GRUPO'] = arquivos['PATH'].apply(self.__obter_grupo)

      try:
        errors = arquivos.groupby(arquivos['GRUPO']).get_group('')['NAME'].tolist()
      except:
        errors = []

      if len(errors) > 0:
        errors = '\n'.join(errors)
        mensageria.falha(f'\n>>>> Os seguintes arquivos têm localização irregular:\n {errors}')
        return

      # Converte dados em dicionário
      self.__resumo = {}
      self.__resumo['processos'] = self.__obter_dic_processos(arquivos)
      self.__resumo['resumo'] = self.__obter_list_resumo(arquivos, grupos)
      self.__ajustar_padroes()
      self.__processos = len(self.__resumo['processos'].keys())
      self.__grupos = []
      for item in self.__resumo['resumo']:
        dic = {}
        dic['nome'] = item['titulo']
        dic['processos'] = len(item['processos'])
        self.__grupos.append(dic)

      usuarios = self.__obter_usuarios_fake()
      self.__resumo_antigo = {}
      self.__resumo_antigo['processos'] = self.__obter_dic_processos_antigo(arquivos)
      self.__resumo_antigo['usuarios'] = usuarios
      self.__resumo_antigo['sessao'] = self.__obter_sessao_fake(usuarios)
      self.__resumo_antigo['resumo'] = self.__obter_list_resumo(arquivos, grupos)
      self.__resumo_antigo['titulos'] = self.__obter_titulos_fake()

      self.__votos = arquivos
      self.__pastas = pastas

      # Preenche valores vazios de  

    else:
      mensageria.falha(f'\n>>>> O arquivo {nome_arquivo} não é um csv')

    os.remove(nome_arquivo)

  def has_data_frame(self):
    return not self.__votos is None and not self.__pastas is None

  def get_name(self):
    if not self.has_data_frame():
      mensageria.falha('\n>>>> NÃO HÁ DADOS CARREGADOS')
      return
    return f'{self.__cadeira}{self.__data}_{self.__complemento}'

  def get_data_frame(self):
    if not self.has_data_frame():
      mensageria.falha('\n>>>> NÃO HÁ DADOS CARREGADOS')
      return
    return self.__data_frame

  def show_votos(self):
    if not self.has_data_frame():
      mensageria.falha('\n>>>> NÃO HÁ DADOS CARREGADOS')
      return
    print()
    mensageria.normal('\n>>>> VOTOS')
    display(HTML(self.__votos.head().to_html()))
    mensageria.normal('\n>>>> TIPOS')
    for col in self.__votos.columns:
      print(f'{col}: {self.__votos[col].dtype}')

  def show_pastas(self):
    if not self.has_data_frame():
      mensageria.falha('\n>>>> NÃO HÁ DADOS CARREGADOS')
      return
    print()
    mensageria.normal('\n>>>> PASTAS')
    display(HTML(self.__pastas.head().to_html()))
    mensageria.normal('\n>>>> TIPOS')
    for col in self.__pastas.columns:
      print(f'{col}: {self.__pastas[col].dtype}')

  def show_resumo(self):
    if not self.has_data_frame():
      mensageria.falha('\n>>>> NÃO HÁ DADOS CARREGADOS')
      return
    mensageria.sucesso('\n>>>> DADOS OBTIDOS:')
    mensageria.sucesso(f'Total de processo: {self.__processos}')
    mensageria.sucesso('Grupos:')
    for grupo in self.__grupos:
      nome = grupo['nome']
      processos = grupo['processos']
      mensageria.sucesso(f'{nome}: {processos} processos')

  def download(self):
    if not self.has_data_frame():
      mensageria.falha('\n>>>> NÃO HÁ DADOS CARREGADOS')
      return
    nome = f'{self.get_name()}.json'
    if os.path.isfile(nome):
      os.remove(nome)
    with open(nome, 'w', encoding='utf-8') as file:
      file.write(json.dumps(self.__resumo, indent=2))
    files.download(nome)

  def download_antigo(self):
    if not self.has_data_frame():
      mensageria.falha('\n>>>> NÃO HÁ DADOS CARREGADOS')
      return
    nome = f'{self.get_name()}.json'
    if os.path.isfile(nome):
      os.remove(nome)
    with open(nome, 'w', encoding='utf-8') as file:
      file.write(json.dumps(self.__resumo_antigo, indent=2))
    files.download(nome)

print()
mensageria.sucesso('\n>>>> Ferramentas carregadas com sucesso!')

In [ ]:
#@title **CARREGAR DADOS DO SHAREPOINT** { vertical-output: true }

##
## Instanciação
##
sharepoint = Sharepoint.instance()
sharepoint.load()
sharepoint.show_resumo()

In [ ]:
#@title **BAIXAR `JSON` (ANTIGO)**

sharepoint.download_antigo()

## **AVANÇADO**

In [ ]:
#@title **BAIXAR `JSON`**

sharepoint.download()

In [ ]:
#@title **VER DATAFRAMES** { vertical-output: true }

##
## Instanciação
##
sharepoint.show_votos()
sharepoint.show_resumo()